In [112]:
%run get_target_toponyms.ipynb
%run lit_utility_functions_2025.ipynb

from datetime import datetime, UTC
import nltk
from nltk.util import ngrams
import re
import warnings

verbose=True

In [78]:
test_pts_path = os.path.join(datdir, 'test_gages', 'test_gages.shp')

wbd_path = os.path.join(nhd_dir, 'WBD_National_GDB.gdb')
hu12_parquet = os.path.join(nhd_dir, 'wbd_hu12list.parquet')
gadm_path = os.path.join(gadm_dir, 'gadm_410-levels.gpkg')

In [80]:
#Get hydronyms from NHD
all_huc12_list = create_huc12_list(wbd_path, hu12_parquet)

target_huc_pd = get_matching_NHD_HU(
    in_wbd_path=wbd_path,
    in_vector=test_pts_path,
    in_refids_parquet=hu12_parquet,
    hull=True,
    sjoin_predicate='intersects'
)

In [125]:
target_nhd_hydronyms = get_nhd_hydronyms(
    in_hucs=target_huc_pd,
    in_wbd_path=wbd_path,
    out_dir = os.path.join(nhd_dir, 'nhdplus_hr'),
    huc_range=[2, 4, 6],
    flatten=True,
    verbose=False
)

Getting NHD basin names


In [82]:
target_gadm_lev1 = get_gadm_lev1_dict(
    in_gadm_path=gadm_path,
    in_vector=test_pts_path,
    hull=True,
    sjoin_predicate='intersects'
)

In [117]:
#Make a flattened set
target_toponyms_list = target_nhd_hydronyms.copy()
target_toponyms_list.update(target_gadm_lev1['NAME_1'].tolist())

#Serialize toponyms
target_toponyms_pkl = os.path.join(
    resdir, 
    f"target_toponyms_umrb_{datetime.now(UTC).strftime('%Y%m%d%H%M')}.pkl"
)
with open(target_toponyms_pkl, 'wb') as f:
    pickle.dump(target_toponyms_list, f)

In [142]:
print(len(target_toponyms_list))

909
